<a href="https://colab.research.google.com/github/HakkiAkut/ai-ml-algorithms/blob/master/Perceptron/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Drive added

In [ ]:
%cd /content/drive/MyDrive/ML/perceptron/

/content/drive/MyDrive/CSE463/Assignment01


Importing pandas, numpy, sklearn.preprocessing.LabelEncoder, sklearn.preprocessing.OneHotEncoder, sklearn.model_selection.train_test_split

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

Receiving data from csv files

In [ ]:
iris = pd.read_csv("iris.csv") # receive data from iris.csv
iris_shuffled = iris.sample(frac=1)  # shuffle

iris_modified = pd.read_csv("irismodified.csv") # receive data from irismodified.csv
iris_modified_shuffled = iris_modified.sample(frac=1)  # shuffle

Changing categorical values to One-Hot Encoded values.

In [ ]:
def one_hot_encoding(data):
  labelEncoder= LabelEncoder() # label encoder
  oneHotEncoder = OneHotEncoder(sparse=False) # one hot encoder
  labels_int = labelEncoder.fit_transform(data)
  labels_int = labels_int.reshape(len(labels_int), 1)
  labels_onehot = oneHotEncoder.fit_transform(labels_int)
  return labels_onehot

Data splitting to labels and features

In [ ]:
def data_split(data, labelColumn):
  labels= data.iloc[0:,labelColumn].values # takes last value only 
  labels= one_hot_encoding(labels)
  features = data.iloc[0:,0:labelColumn].values # takes features which is 1th to 4th columns for iris.csv 
  return features, labels

Multi-class Perceptron Training function

In [ ]:
def multiclass_training(features, labels, lr=0.001, iteration=5):
  weights=[]
  bias=[]
  # initializing weights and bias. Since, One hot encode for iris.csv has are 3 colums, we initiliaze weights and bias as list of 3.  
  for j in range(len(labels[0])):
    weights.append(np.zeros(features.shape[1])) # weights = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] for iris.csv
    bias.append(0.0) # bias = [0.0, 0.0, 0.0] for iris.csv

  # training the same dataset for 'iteration' times
  for i in range(iteration):
    index = 0;
    #training each row
    for sample in features:
      pred = multiclass_prediction(sample, weights, bias) # returns as [1 0 0] or [0 1 0] or [0 0 1]
      # updating weights and bias for each column
      for k in range(len(bias)):
        delta_weights= lr*(labels[index][k]- pred[k])
        weights[k]+= delta_weights*sample
        bias[k] += delta_weights
      index+=1
  return weights,bias

Multi-class Prediction Function

In [ ]:
def multiclass_prediction(features, weights, bias): #looks like => weights = [[x x x x],[x x x x],[x x x x]] bias = [x x x x] for iris.csv
  calculated_inputs=[]
  # predicting for each rows of one hot encoded labels value, it's 3 columns for iris.csv 
  for k in range(len(bias)):
    calc_inp = np.dot(features, weights[k])+bias[k]
    calculated_inputs.append(calc_inp)
  gp=0; 
  # finding greatest probability and changing to one hot encoded type. [0.4, 0.5, 0,3] becomes [0, 1, 0]
  for j in range(len(calculated_inputs)):
    if calculated_inputs[j] > calculated_inputs[gp]:
      gp= j
  pred=np.zeros(len(calculated_inputs))
  pred[gp]= 1
  return pred 

Initializing datasets

In [ ]:
# iris.csv
features1, labels1= data_split(iris_shuffled, 4)  
features_train, features_test, labels_train, labels_test = train_test_split(features1, labels1, test_size =0.2, random_state=42) # split into training set and test set
# irismodified.csv
features_modified, labels_modified = data_split(iris_modified_shuffled, 4) 
features_modified_train, features_modified_test, labels_modified_train, labels_modified_test = train_test_split(features_modified, labels_modified, test_size =0.2, random_state=42) 

Checking the predictions are true or false

In [ ]:
def check_prediction_list(prediction, result):
  checked = prediction == result # becomes [True True True] or [True False False] type of object
  if type(checked)== bool:
    print(checked,prediction,result)
    if checked == False:
      return 1
    else:
      return 0
  else:
    if False in checked: # it has False value, recorded as false 
      return 1
    else:
      return 0

Calculating the metrics



In [ ]:
def find_metrics(labels,check_list,decision_list):
  n = len(check_list)
  error=0
  label_num= np.zeros(len(labels[0])) # number of real values
  label_true= np.zeros(len(labels[0])) # number of true predicts
  label_pred= np.zeros(len(labels[0])) # number of predicted values 
  precision_list= np.zeros(len(labels[0]))
  recall_list= np.zeros(len(labels[0]))
  # calculating number of real values, true predicts, predicted values 
  for k in range(n):
    index = find_index(labels[k])
    label_num[index] = label_num[index] + 1
    if check_list[k] == 0:
      label_true[index] = label_true[index] + 1
      label_pred[index] = label_pred[index] + 1
    else:
      index_choosed = find_index(decision_list[k])
      label_pred[index_choosed] = label_pred[index_choosed] + 1
      error +=1
  # calculating each precision and recall  
  for j in range(len(label_num)):
    recall_list[j]= label_true[j]/label_num[j]
    precision_list[j]= label_true[j]/label_pred[j]
  # calculating average precision and recall 
  recall_avg= sum(recall_list)/len(recall_list)
  precision_avg= sum(precision_list)/len(precision_list)
  # printing the metrics
  print(f"average accuracy {(n-error)/n}") 
  print(f"average recall {recall_avg}")
  print(f"average precision {precision_avg}")  

def find_index(label):
  index=0
  for k in range(len(label)):
    if label[k] == 1:
      return k 

Part 1 (%80 of iris.csv for training and %20 of iris.csv for test) iris dataset has 3 classes.

In [ ]:
# train with %80 percent of iris.csv data
weights_trained, bias_trained = multiclass_training(features_train, labels_train, lr=0.001, iteration=500)

check_list= []
predictions= []
error_num= 0
# test with remaining
zipper = zip(features_test, labels_test)
for sample , result in zipper:
  prediction_final = multiclass_prediction(sample, weights_trained, bias_trained)
  predictions.append(prediction_final)
  error = check_prediction_list(prediction_final, result)
  check_list.append(error)
  error_num= error_num + error

find_metrics(labels_test,check_list,predictions)

average accuracy 0.9333333333333333
average recall 0.9393939393939394
average precision 0.9393939393939394


I used accuracy, recall and, precision 

average accuracy 0.9333333333333333,

average recall 0.9393939393939394,

average precision 0.9393939393939394,

With these metrics, I would say that it's a really good model

[I used this formulas for finding precision and recall for multiclass prediction](https://medium.com/data-science-in-your-pocket/calculating-precision-recall-for-multi-class-classification-9055931ee229)

Part 2 (%80 of irismodified.csv for training and %20 of irismodified.csv for test) irismodified  dataset hast 2 classes

In [ ]:
# train with %80 percent of irismodified.csv data
weights_trained2, bias_trained2 = multiclass_training(features_modified_train, labels_modified_train, lr=0.001, iteration=500)
check_list= []
predictions= []
error_num= 0
# test with remaining
zipper = zip(features_modified_test, labels_modified_test)
for sample , result in zipper:
  prediction_final = multiclass_prediction(sample, weights_trained2, bias_trained2)
  predictions.append(prediction_final)
  error = check_prediction_list(prediction_final, result)
  check_list.append(error)
  error_num = error_num + error

find_metrics(labels_modified_test,check_list,predictions)

average accuracy 0.9
average recall 0.898989898989899
average precision 0.898989898989899


I used accuracy, recall and, precision 
average accuracy 0.9,
average recall 0.898989898989899,
average precision 0.898989898989899,
With these metrics, I would say that it's a good model